In [13]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import sqlalchemy

from xml.etree import ElementTree

import mercante

engine = sqlalchemy.create_engine(
    'mysql+mysqlconnector://ivan@localhost:3306/mercante')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
lista_arquivos = os.listdir(mercante.MERCANTE_DIR)
for arquivo in lista_arquivos:
    xtree = ElementTree.parse(os.path.join(mercante.MERCANTE_DIR, arquivo))
    xroot = xtree.getroot()
    objetos = []
    for node in xroot:
        classe = mercante.classes.get(node.tag)
        if classe:
            # print(node.tag, node.text)
            objeto = classe()
            objeto._parse_node(node)
            # objeto = Conhecimento()
            # objeto._parse_node(node)
            # print(objeto)
            objetos.append(objeto._to_dict())
    df = pd.DataFrame(objetos)
    try:
        df.to_sql(node.tag, engine, if_exists='append')
    except Exception as err:
        print('Erro ocorrido no arquivo %s.' % arquivo)
        print(err, exc_info=True)
    